# Calculate POT

Adapting from Marina's POT code

This code teaches you how to calculate the POT for a given sample. There is an option of applying cuts on the sample before calculating the POT, which is important for beam-on, for example, as we are interested in the POT after applying the good-quality check and in a specific neutrino mode (for run 1 it means to select the runs < 6748). So make sure you are applying all the necessary cuts before calculating the POT, as the POT is used as an input for the analysis.

Proper wiki link to running POT calculation script (zarko's script): https://cdcvs.fnal.gov/redmine/projects/uboone-physics-analysis/wiki/NuMI_Documentation#How-to-do-NuMI-POT-counting

In [1]:
from IPython.core.display import display, HTML # fix window size
display(HTML('<style>.container {width:100% !important;}</style>'))
import pandas as pd
import uproot3 as uproot
import numpy as np

/var/folders/61/z_mpdl8x31dcjrks2m91kynw0000gp/T/ipykernel_78188/567715129.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython.display
  from IPython.core.display import display, HTML # fix window size


In [2]:
def gen_run_subrun_list(input_file, name_list):
    # Get useful variables
    T_eval = uproot.open(input_file)['wcpselection/T_eval']
    df_eval = T_eval.pandas.df(['run', 'subrun'], flatten=False)
    df_eval.drop_duplicates(inplace=True) 
    np.savetxt(name_list, df_eval.values, fmt='%d')
    
# --- keep only good-quality events
def apply_good_quality_check(df_in, file):
    # Create dataframe with the good-quality runs you printed above
    df_good_runs = pd.read_csv(file, sep=", ", header=None, engine='python')
    df_good_runs = df_good_runs.T # transpose so it's a column dataframe
    df_good_runs.rename(columns={0:'run'}, inplace=True) # rename column to match with the one below
    
    # apply the cut
    df = pd.merge(df_in, df_good_runs, how='left', indicator=True)
    df_good = df[df._merge=='both'] # good-quality runs in input sample
    df_bad = df[df._merge!='both'] # bad-quality runs in input sample
    
    # uncomment if you want to select GOOD-QUALITY events
    df_final = df_good.drop(columns="_merge") # remove column "merge" only used for distinguishin good/bad
    df_final.drop_duplicates(inplace=True) # remove duplicates
    
    bad_quality = False # false=keep good-quality, true=keep bad-quality
    if(bad_quality):
        df_final = df_bad.drop(columns="_merge")
        df_final.drop_duplicates(inplace=True)
    
    return df_final

# Run 3 Beam-On

This part is necessary for beam-on only. As we want to apply the good-quality check, we need the list of good-quality runs that will be used as a filter to compare to the runs present in the sample and keep only the good-quality ones. This part of the code you have to do manually on the terminal. First, create all the necessary tickets to run samweb commands:

> /cvmfs/uboone.opensciencegrid.org/bin/shell_apptainer.sh <br />
> source /cvmfs/uboone.opensciencegrid.org/products/setup_uboone.sh <br />
> setup uboonecode v09_32_00 -q e20:prof <br />
> setup sam_web_client <br />
> voms-proxy-init -noregen -rfc -voms fermilab:/fermilab/uboone/Role=Analysis <br />
> kx509 <br />

Which can quickly be ran as:

> cd /exp/uboone/app/users/wwang/POT_check <br />
> source setup_shell.sh <br />
> source setup_samweb.sh <br />

<!-- Then, run the samweb command that will print the list of run/subrun for the good-quality ones:

> cd /uboone/app/users/mguzzo/antinue <br />
> samweb describe-definition goodruns_mcc9_run1_high_lifetime > calculate_pot_run1_beamon_goodquality.list

Unfortunately the list generated above is not in the correct format, so let's work on this so it is in the format that the code below needs it to be. You have to do a few things:
- ( 1 ) delete the header of the file, everything that is not run numbers
- ( 2 ) manually remove all the new lines and make it into a single line list
- ( 3 ) make sure all run numbers have ", " in front of it

Once you're done with the steps above, you'll have a run_list_run1_beamon_original.list in the correct format to be used by the code below: -->

In [3]:
# ===================== #
#   INPUT INFORMATION   #
# ===================== #

# you need:
# 1) the good-quality run list generated above (samdef: goodruns_mcc9_run1_high_lifetime)
# 2) the checkout file

inFile = './ROOT_files/checkout_data_numi_run3_RHC.root'

# ======================== #
#   POT OF ENTIRE SAMPLE   #
# ======================== #

# save the original run/subrun list, without applying any cut
T_eval = uproot.open(inFile)['wcpselection/T_eval']
df_eval = T_eval.pandas.df(['run', 'subrun'], flatten=False)
# df_eval.drop_duplicates(inplace=True) 
#np.savetxt("/uboone/app/users/mguzzo/antinue/run_subrun_list_run1_beamon_original.list", df_eval.values, fmt='%d')

# ======================== #
#   APPLY CUTS ON SAMPLE   #
# ======================== #

#df_filtered = apply_good_quality_check(df_eval, 'goodruns_mcc9_run3_hardcoded.list')
#np.savetxt("/uboone/app/users/mguzzo/antinue/run_subrun_list_run1_beamon_filtered.list", df_filtered.values, fmt='%d') # save list of good runs

# --- select before/after trigger change.
# df_filtered_before = df_filtered[df_filtuered.run < 16880]
# df_filtered_after = df_filtered[df_filtered.run >= 16880]
#np.savetxt("./calc_data_pot_run_3.txt", df_filtered.values, fmt='%d') # save list of good run
# np.savetxt("./calc_data_pot_run_3b_after_filtered.txt", df_filtered_after.values, fmt='%d')
gen_run_subrun_list(inFile, "./calc_data_pot_run_3.txt")

There is the need to first upload the .txt file up onto gpvm, and then go from there

Open a terminal:

> cd /exp/uboone/app/users/wwang/POT_check <br />
> source setup_shell.sh <br />
> source setup_samweb.sh <br />
> unsetup python <br />
> setup python v2_7_3 <br />
> /exp/uboone/app/users/zarko/getDataInfo.py -v3 --format-numi --prescale --run-subrun-list /exp/uboone/app/users/wwang/POT_check/calc_data_pot_run_3.txt <br />

---

```
Read 233725 lines from /exp/uboone/app/users/wwang/POT_check/calc_data_pot_run_3.txt
           EXT         Gate1        EA9CNT        tor101        tortgt   EA9CNT_wcut   tor101_wcut   tortgt_wcut
   109120846.0     9650960.0     9640633.0     4.583e+20     4.563e+20     9431541.0     4.581e+20     4.562e+20

	EXT_unbiased_PrescaleAlgo                          2182416.920000
	NUMI_unbiased_PrescaleAlgo                         154415.360000
	EXT_HSN_c0_FEMBeamTriggerAlgo                      21824169.200000
	EXT_BNBwin_2017Dec_SWTrigger5PE_FEMBeamTriggerAlgo 105285540.000000
	NUMI_2017Dec_SWTrigger8_5PE_FEMBeamTriggerAlgo     2448.000000
	EXT_HSN_c1_FEMBeamTriggerAlgo                      7429.200000
	EXT_NUMIwin_FEMBeamTriggerAlgo                     19096148.050000
	NUMI_2018May_FEMBeamTriggerAlgo                    1802194.000000
	EXT_NUMIwin_2018May_FEMBeamTriggerAlgo             4459695.800000
	NUMI_FEMBeamTriggerAlgo                            9650960.000000
	NUMI_2017Dec_SWTrigger9PE_FEMBeamTriggerAlgo       963.000000
	EXT_BNBwin_FEMBeamTriggerAlgo                      109120846.000000
	EXT_NUMIwin_2017Dec_SWTrigger8_5PE_FEMBeamTriggerAlgo 2685.375000
	EXT_NUMIwin_2017Dec_SWTrigger9PE_FEMBeamTriggerAlgo 1474.200000
	NUMI_2017Dec_SWTrigger8PE_FEMBeamTriggerAlgo       2114.000000
	EXT_NUMIwin_2017Dec_SWTrigger8PE_FEMBeamTriggerAlgo 2340.975000
```

In [4]:
# Run 3 Beam-On
run3_data_pot = 4.562e+20 # (tortgt_wcut)
print("Run 3 DATA POT = %.2e" % run3_data_pot)

Run 3 DATA POT = 4.56e+20


# Run 3 EXT

To calculate the EXT POT, first do the same step as the beam-on, produce run/subrun list for EXT beam-off file and run it through zarko's script

In [5]:
# ===================== #
#   INPUT INFORMATION   #
# ===================== #

inFile_run3_ext = './ROOT_files/checkout_data_extnumi_run3.root'

# save the run/subrun list, through obtaining the T_eval tree's run/subrun NTuples
T_eval = uproot.open(inFile)['wcpselection/T_eval']
df_eval = T_eval.pandas.df(['run', 'subrun'], flatten=False)

# save the run/subrun list with the pre-written function
gen_run_subrun_list(inFile_run3_ext, "./calc_ext_pot_run_3.txt")

Open a terminal:

> cd /exp/uboone/app/users/wwang/POT_check <br />
> source setup_shell.sh <br />
> source setup_samweb.sh <br />
> unsetup python <br />
> setup python v2_7_3 <br />
> /exp/uboone/app/users/zarko/getDataInfo.py -v3 --format-numi --prescale --run-subrun-list /exp/uboone/app/users/wwang/POT_check/calc_ext_pot_run_3.txt <br />

---

```
Read 337088 lines from /exp/uboone/app/users/wwang/POT_check/calc_ext_pot_run_3.txt
           EXT         Gate1        EA9CNT        tor101        tortgt   EA9CNT_wcut   tor101_wcut   tortgt_wcut
   173972943.0     9887458.0     9885888.0     4.535e+20     4.516e+20     9334387.0     4.533e+20     4.514e+20

	EXT_unbiased_PrescaleAlgo                          3573973.980000
	NUMI_unbiased_PrescaleAlgo                         158199.328000
	EXT_HSN_c0_FEMBeamTriggerAlgo                      34794588.600000
	EXT_BNBwin_2017Dec_SWTrigger5PE_FEMBeamTriggerAlgo 169376466.000000
	NUMI_2017Dec_SWTrigger8_5PE_FEMBeamTriggerAlgo     2404.000000
	EXT_HSN_c1_FEMBeamTriggerAlgo                      9293.000000
	EXT_BNBwin_2017Dec_v2_FEMBeamTriggerAlgo           12976.000000
	EXT_NUMIwin_FEMBeamTriggerAlgo                     30445265.025000
	NUMI_2018May_FEMBeamTriggerAlgo                    1859774.000000
	EXT_NUMIwin_2018May_FEMBeamTriggerAlgo             13740405.875000
	NUMI_FEMBeamTriggerAlgo                            9887458.000000
	EXT_BNBwin_2017Dec_v1_FEMBeamTriggerAlgo           16611.000000
	EXT_BNBwin_2017Dec_v3_FEMBeamTriggerAlgo           14097.000000
	NUMI_2017Dec_SWTrigger9PE_FEMBeamTriggerAlgo       1294.000000
	EXT_BNBwin_FEMBeamTriggerAlgo                      173972943.000000
	EXT_NUMIwin_2017Dec_SWTrigger8_5PE_FEMBeamTriggerAlgo 2637.775000
	EXT_NUMIwin_2017Dec_SWTrigger9PE_FEMBeamTriggerAlgo 2855.650000
	NUMI_2017Dec_SWTrigger8PE_FEMBeamTriggerAlgo       2383.000000
	EXT_BNBwin_2017Dec_v4_FEMBeamTriggerAlgo           15807.000000
	EXT_NUMIwin_2017Dec_SWTrigger8PE_FEMBeamTriggerAlgo 2637.950000
```

In [11]:
run3_EXT_EA9CNT_wcut = 9334387.0 #9431541.0
run3_EXT_NUMIwin_FEMBeamTriggerAlgo = 30445265.025000
run3_ext_pot = run3_data_pot/(run3_EXT_EA9CNT_wcut/run3_EXT_NUMIwin_FEMBeamTriggerAlgo)
print("Run 3 EXT POT = %.2e" % run3_ext_pot)

Run 3 EXT POT = 1.49e+21
